In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from tqdm import tqdm

chrome_options = webdriver.ChromeOptions()
#headless means running chrome with chrome.exe
chrome_options.add_argument('--headless')

chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [51]:
# wd = webdriver.Chrome(options=chrome_options, executable_path='C:/chromedriver.exe')

<ipython-input-51-f68ab8f240ff>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(options=chrome_options, executable_path='C:/chromedriver.exe')


In [3]:
data = pd.read_excel('Movie List.xlsx')

In [5]:
movie_names = data['Movie']
movie_names

0         Love by Drowning
1      Mickey and the Bear
2                 Hustlers
3              Oppenheimer
4            Another Round
              ...         
549              12 Strong
550     The Shape of Water
551               The Post
552     I Can Only Imagine
553     Fifty Shades Freed
Name: Movie, Length: 554, dtype: object

In [6]:
def movie_page(prompt):
    wd = webdriver.Chrome(options=chrome_options)

    url = 'https://www.rottentomatoes.com/'

    wd.get(url)
    search = WebDriverWait(wd, 1).until(
                    EC.element_to_be_clickable(
                        wd.find_element(by=By.XPATH, value='/html/body/div[3]/rt-header/search-algolia/search-algolia-controls/input')))
    search.send_keys(prompt)
    search.send_keys(Keys.RETURN)
    
    return wd

In [7]:
# rotten_movie_links = {}

# for i in tqdm(movie_names):
#     wd = movie_page(i)
#     time.sleep(3)
#     link = wd.find_element(by=By.TAG_NAME, value='search-page-media-row')
    
#     rotten_movie_links[i] = link.find_element(by=By.TAG_NAME, value='a').get_attribute('href')
#     # print(i)



In [8]:
# rotten_df = pd.DataFrame(list(rotten_movie_links.items()), columns=["Movie", "URL"])
# display(rotten_df)

In [9]:
rotten_links = pd.read_csv('Rotten links.csv')

In [10]:
rotten_links

,Movie,URL
0,Love by Drowning,https://www.rottentomatoes.com/m/love_by_drowning
1,Mickey and the Bear,https://www.rottentomatoes.com/m/mickey_and_th...
2,Hustlers,https://www.rottentomatoes.com/m/hustlers_2019
3,Oppenheimer,https://www.rottentomatoes.com/m/oppenheimer_2023
4,Another Round,https://www.rottentomatoes.com/m/another_round
...,...,...
549,Lady Bird,https://www.rottentomatoes.com/m/lady_bird
550,Gotti,https://www.rottentomatoes.com/m/gotti_2018
551,The Rider,https://www.rottentomatoes.com/m/the_rider
552,Nobody’s Fool,https://www.rottentomatoes.com/m/nobodys_fool_...


In [35]:
rotten_top_critic_links = {}

for i,url in tqdm(zip(rotten_links['Movie'], rotten_links['URL']), total=len(rotten_links)):
    wd = webdriver.Chrome(options=chrome_options)
    wd.get(url)
    
    try:
        link = wd.find_element(By.XPATH, '//*[@id="critics-reviews"]/div/div/div[1]/a[2]').get_attribute('href')
        rotten_top_critic_links[i]=link
    except:
        rotten_top_critic_links[i] = 'ERROR'

  1%|▋                                                                               | 5/554 [01:51<3:23:18, 22.22s/it]


KeyboardInterrupt: 

In [38]:
rotten_top_critic_links

{'Love by Drowning': 'ERROR',
 'Mickey and the Bear': 'https://www.rottentomatoes.com/m/mickey_and_the_bear/reviews?type=top_critics',
 'Hustlers': 'https://www.rottentomatoes.com/m/hustlers_2019/reviews?type=top_critics',
 'Oppenheimer': 'https://www.rottentomatoes.com/m/oppenheimer_2023/reviews?type=top_critics',
 'Another Round': 'https://www.rottentomatoes.com/m/another_round/reviews?type=top_critics'}

In [39]:
top_df = pd.DataFrame(list(rotten_top_critic_links.items()), columns=["Movie", "URL"])

In [41]:
def extract_comments(wd):
    comments=[]
    for i in wd.find_elements(By.CLASS_NAME, value='review-text'):
        comments.append(i.text)
    return comments
        
def click_next(wd):
    WebDriverWait(wd, 3).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="reviews"]/div[1]/rt-button[2]'))).click()

def get_reviews(url, comments):
    wd = webdriver.Chrome(options=chrome_options)
    wd.get(url)
    comments.extend(extract_comments(wd))

    while True:
        try:
            click_next(wd)
            time.sleep(3)
            comments.extend(extract_comments(wd))
        except:
            break
    return comments

In [48]:
all_comments={}

for i,url in tqdm(zip(top_df['Movie'], top_df['URL']), total=len(top_df)):
    if url=='ERROR':
        all_comments[i] = ['ERROR']
        continue
    else:
        comments=[]
        all_comments[i] = get_reviews(url,comments)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:20<00:00, 16.13s/it]
